In [1]:
import tensorflow as tf 

In [2]:
import os
os.chdir('../')
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]= "URI=https://dagshub.com/PyBeginner1/Deep-CNN_Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="PyBeginner1"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ef0c7d067cbd1be8ceddac7de8d97dc9a7cf7748"

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    mlflow_uri : str
    all_params : dict
    params_image_size: list
    params_batch_size: int

In [ ]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH, 
                params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e
        
    def get_evaluation_config(self) -> EvaluationConfig:
        try:
            eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data= "artifacts/data_ingestion/PetImages",
                mlflow_uri= "https://dagshub.com/PyBeginner1/Deep-CNN_Classifier.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size = self.params.BATCH_SIZE
            )

            return eval_config
        except Exception as e:
            raise e

In [ ]:
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"Loss": self.score[0], "Accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [8]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config = eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1864s 4s/step - loss: 2.1220 - accuracy: 0.8325


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'S:\\'